# Pré-traitement des données d'entrée du modèle

In [ ]:
import numpy as np
import pandas as pd
from sklearn import set_config

set_config(transform_output="pandas")

## Normalisation

Certains modèles font des hypothèses sur la distribution des données d'entrée. En particulier ils peuvent supposer que la variance de toutes les variables sont du même ordre de grandeur pour fonctionner correctement.  
C'est en particulier le cas des machines à vecteurs de support (*support vector machines* ou SVM) qui seront utilisées dans cette section.

Les données support de cette partie sont des mesures réalisées sur des tumeurs afin de détecter un éventuel cancer du sein.  
La description du jeu de données est fournie dans l'élément `DESCR` du dictionnaire (`data`).  
En particulier, cette description donnes des informations de dispersion des mesures (le min et le max pour chaque variable).

In [ ]:
from sklearn import datasets

data = datasets.load_breast_cancer()
X_np, y = data.data, data.target_names[data.target]
X = pd.DataFrame(X_np, columns=data["feature_names"])
X.describe()

In [ ]:
print(data["DESCR"])

Comme souvent en apprentissage automatique, le jeu de données annoté est séparé en deux : un jeu de données d'entrainement (75%) et un jeu de données de validation (25%), servant à évaluer le pouvoir prédictif du modèle.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

**Question** : Entraîner un classificateur à vecteurs de support ([classe `SVC` de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)) sur le jeu de données d'entraînement, puis afficher les deux matrices de confusion : une pour les données de test et une pour les données d'entrainement.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt

# Éventuellement pour vous simplifier la vie, il n'est pas obligatoire d'utiliser cette fonction.
def plot_confusion_matrix(y_test_true, y_test_pred, y_train_true, y_train_pred):
    """Display confusion matrices side-by-side."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
    for y_true, y_pred, ax, title in [
        (y_test_true, y_test_pred, ax1, "Test data"),
        (y_train_true, y_train_pred, ax2, "Train data"),
    ]:
        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, ax=ax)
        ax.set_title(title)

Nous allons ensuite tenter d'améliorer la performance du modèle en normalisant les données d'entrée.

**Question** : Est-il souhaitable de réaliser cette normalisation sur l'ensemble du jeu de données original ?

**Question** : Réaliser une transformation des données d'entrée pour les mettre à une échelle [0, 1] ([classe `MinMaxScaler` de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)).

In [ ]:
from sklearn.preprocessing import MinMaxScaler

**Question** : Quelles sont les plages de valeurs des données d'entrainement transformées ?  
Sont-elles dans l'intervalle [0, 1] ?

**Question** : Quelles sont les plages de valeurs des données de test transformées ?  
Sont-elles dans l'intervalle [0, 1] ?

**Question** : Réaliser un nouvel entraînement sur les données transformées et afficher les matrices de confusion correspondantes.  
Le résultat a-t-il changé par rapport au modèle précédent, entraîné sur les données brutes ?

**Question** Reprendre les quatre questions précédentes en limitant les valeurs de sortie du prétraitement : `MinMaxScaler(clip=True)`.  
Cette modification a-t-elle un impact sur la précision du modèle ?

## Séquences de traitements (*pipelines*)

Dans la section précédente, nous avons réalisé une partie des prétraitements sur le jeu d'entraînement (`fit` et `transform`) et une autre sur le jeu de validation (`transform` uniquement), puis la même chose pour le classificateur (`fit` et `predict`, ou uniquement `predict`). Avec scikit-learn, il est possible de définir des [séquences de traitements](https://scikit-learn.org/stable/modules/compose.html) dont l'entraînement est ensuite commun.  
Cela permet en particulier d'éviter que des informations statistiques issues des données de test ne soient utilisées pour l'entraînement. Ce sera particulièrement utile dans la suite du cours pour réaliser une validation croisée ou optimiser les hyper-paramètres d'un modèle.

**Question** : Définir une pipeline intégrant la normalisation et la classification, puis vérifier que vous obteniez bien les mêmes résultats que précédemment.

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('classifier', SVC())
])

**Question** : Vérifier que la standardisation ne s'est bien basée que sur les données d'entraînement.  
Pour cela, il est possible d'extraire un estimateur correspondant à une sous-chaîne de traitement. Ici, on prendra l'unique étape de prétraitement, c'est-à-dire `pipeline[0]`, ou, plus généralement, `pipeline[:-1]` (toutes les étapes sauf la dernière).

In [ ]:
preprocessing = pipeline[:-1]
preprocessing.transform(X_test).describe()

## Gestion des valeurs manquantes

Votre assistant a malencontreusement perdu les mesures de valeur moyenne et de pire tumeur pour un quart des patients (il ne lui reste que les mesures d'écart type, soit un tiers des variables d'entrée).

In [ ]:
missing_cols = [col for col in X.columns if "mean" in col or "worst" in col]
print(f"Deleted columns: {missing_cols} ({len(missing_cols)/len(X.columns):.2%} of features)")
X_missing = X.copy()
missing_idx = X_missing.sample(frac=0.25).index
X_missing.loc[missing_idx, missing_cols] = np.nan

In [ ]:
X_missing.iloc[:, :15].describe()

In [ ]:
X_train_m, X_test_m, y_train, y_test = train_test_split(X_missing, y, random_state=123)
print(f"{len(set(X_train_m.index).intersection(missing_idx))/len(X_train_m):.2%} of the train dataset has missing values, "
      f"{len(set(X_test_m.index).intersection(missing_idx))/len(X_test_m):.2%} of the test dataset has missing values.")

La grande majorité des modèles de scikit-learn ne supportent pas les valeurs manquantes (comme généralement en apprentissage automatique).  
**Question** : Vérifier si votre chaîne de traitement est compatible avec les nouvelles données d'entrée.

**Question** : Définir une nouvelle chaine de traitements permettant de prendre en compte l'absence de valeurs renseignées.  
Vous avez pour cela trois grands types de stratégies :
- ignorer les colonnes en question (pour appliquer des traitements spécifiques à des colonnes, vous pouvez utiliser [la classe `ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html))
- remplacer les valeurs manquantes par des valeurs typiques (voir [la classe `SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer))
- remplacer les valeurs manquantes par des valeurs estimées par un autre modèle (par exemple, en se basant sur les [mesures les plus proches](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer

## Données catégorielles

Les modèles d'apprentissage automatique attendent en entrée des vecteurs numériques de taille fixe. Il est donc nécessaire de transformer les entrées catégorielles, surtout si elles sont représentées par des chaînes de caractères.

In [ ]:
X = pd.DataFrame({"pets": ["dog", "cat", "snake", "dog", "goldfish", "dog"]})
X_train = X[:4]
X_test = X[4:]

In [ ]:
display(X_train)
display(X_test)

### One hot encoding

Une manière de réaliser ce type de transformation consiste à générer autant de variables booléennes qu'il existe de catégories. On obtient donc en sortie du traitement, pour *n* catégories, un vecteur de taille *n*, avec la valeur 1 pour la colonne correspondant à la catégorie de l'observation et 0 partout ailleurs.  
Cette transformation est désignée comme un encodage un parmi n, ou *one-hot encoding* en anglais.

La bibliothèque pandas propose [la méthode `get_dummies`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) pour réaliser ce type de traitement.

In [ ]:
pd.get_dummies(X)

**Question** : Pourquoi n'est-il pas correct de réaliser ce type de prétraitement globalement ?

**Question** : Utiliser [la classe `OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) (avec, entre autres, l'option `sparse_output=False`) pour réaliser le prétraitement.  
Visualiser les données d'entrainement et de test après transformation.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

### Ordinal encoder

Pour transformer les valeurs catégorielles en valeurs numériques, il est aussi possible de passer par une représentation entière, discrète elle aussi, à l'aide de [la classe `OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html).  
Attention cependant à son utilisation, car de nombreux modèles vont alors supposer qu'il existe un ordre intrinsèque aux catégories, ce qui peut dégrader significativement leur performance.

**Question** : Réaliser un encodage ordinal des données catégorielles et visualiser les données après transformation.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

### MultiLabelBinarizer

Dans certains cas, les catégories peuvent ne pas avoir de valeur unique. C'est-à-dire que chaque observation peut être associée à zéro, une ou plusieurs catégories.

In [ ]:
X = pd.DataFrame({"pets": [{"cat", "dog"}, {"cat"}, {"snake", "dog"}, {"unicorn", "cat"}, {"goldfish"}, {"cat", "dog", "snake"}]})
X_train = X[:3]
X_test = X[3:]

In [ ]:
display(X_train)
display(X_test)

**Question** : Donner (sans automatiser le traitement) un exemple de résultat attendu après transformation.

Scikit-learn n'intègre pas directement d'outil permettant de réaliser ce type de transformation sur les entrées du modèle.  
La transformation la plus proche est un traitement réalisé sur du texte à l'aide de [la classe `CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), où le nombre d'occurrences de chaque mot est compté.  
Il existe quelques différences notables par rapport aux transformations utilisées jusqu'à présent :
- `CountVectorizer` n'accepte qu'une seule colonne en entrée.
- La sortie est une matrice creuse ([*sparse* de scipy](https://docs.scipy.org/doc/scipy/reference/sparse.html)), qui doit être convertie en matrice dense à l'aide de la méthode `todense` pour être compatible avec pandas.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Alternativement, il est (relativement) facile de définir vos propres classes de transformation compatibles avec scikit-learn.

**Question** : Implémenter un transformateur permettant de convertir des listes de catégories en matrices binaires indiquant la présence ou non de chaque catégorie.

## Génération d'entrées complémentaires

Il est possible d'utiliser les transformateurs pour générer des entrées complémentaires. En particulier, scikit-learn n'intègre pas de régression polynomiale, car il est possible d'arriver au même résultat par prétraitement des entrées.  
Dans cette section, nous allons chercher à modéliser des données générées par un polynôme d'ordre deux (avec une seule dimension ici pour simplifier la visualisation, mais cela s'applique aussi à plusieurs dimensions).

In [ ]:
X = np.random.uniform(size=100)
y = 3 - (X - 0.2)**2 + np.random.normal(scale=0.02, size=100)
X = X.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [ ]:
plt.scatter(X_train, y_train, label="train")
plt.scatter(X_test, y_test, label="test")
plt.legend()
plt.show()

**Question** : Utiliser un modèle de régression linéaire (voir [la classe `LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)) pour ce problème de régression.  
Visualiser les prédictions.

In [ ]:
from sklearn.linear_model import LinearRegression

**Question** : Utiliser [la classe `PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) pour générer des combinaisons polynomiales, de degré 2 ou plus, des entrées.  
Visualiser les prédictions.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

## Pipelines ou non ?

**Question** : Parmi toutes les transformations vues dans ce TD, lesquelles peuvent être appliquées au préalable sur l'ensemble du jeu de données ? Pourquoi ?